<a href="https://colab.research.google.com/github/RaghavendraKuratti/BookReviewPrediction/blob/main/BookReviewPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import random

class Sentiment:
	POSITIVE = "POSITIVE"
	NEGATIVE = "NEGATIVE"
	NEUTRAL = "NEUTRAL"
		
class Review:
	# "docstring for Review"
	def __init__(self, text, rate):
		self.text = text
		self.rate = rate
		self.sentiment = self.get_sentiment()

	def get_sentiment(self):
		if self.rate <= 2:
			return Sentiment.NEGATIVE
		elif self.rate == 3:
			return Sentiment.NEUTRAL
		elif self.rate > 3:
			return Sentiment.POSITIVE
		
class ReviewCounter:
	def __init__(self, reviews):
		self.reviews = reviews

	def get_text(self):
		return [x.text for x in  self.reviews]

	def get_y(self):
		return [x.sentiment for x in  self.reviews]

	def evenly_distrubute(self):
		negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
		positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
		positive = positive[:len(negative)]
		self.reviews = negative + positive
		random.shuffle(self.reviews)

import pandas as pd 
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


file_name = "Books_small_10000.json"
reviews = []
with open(file_name) as f:
	for line in f:
		review = json.loads(line)
		reviews.append(Review(review["reviewText"],review["overall"]))
training, testing = train_test_split(reviews, test_size=0.33, random_state=42)

train_cont = ReviewCounter(training)
train_cont.evenly_distrubute()
test_cont = ReviewCounter(testing)
test_cont.evenly_distrubute()

train_x = train_cont.get_text()
train_y = train_cont.get_y()

test_x = test_cont.get_text()
test_y = test_cont.get_y()

vectorizer = CountVectorizer()
train_data_vect = vectorizer.fit_transform(train_x)
test_data_vect = vectorizer.transform(test_x)




from sklearn import svm
clf_svm = svm.SVC(kernel="linear")
clf_svm.fit(train_data_vect,train_y)
result = clf_svm.predict(test_data_vect[0])
# print(result)

from sklearn.tree import DecisionTreeClassifier
clf_dt = DecisionTreeClassifier()
clf_dt.fit(train_data_vect,train_y)
result = clf_dt.predict(test_data_vect[0])
# print(result)

# from sklearn.naive_bayes import GaussianNB
# clf_gnb = GaussianNB()
# clf_gnb.fit(train_data_vect,train_y)
# result = clf_gnb.predict(test_data_vect)
# print(result)

from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression()
clf_lr.fit(train_data_vect,train_y)
result = clf_lr.predict(test_data_vect[0])
# print(result)

# print(clf_svm.score(test_data_vect,test_y))
# print(clf_dt.score(test_data_vect,test_y))
# print(clf_lr.score(test_data_vect,test_y))

from sklearn.metrics import f1_score
# print(f1_score(test_y, clf_svm.predict(test_data_vect), average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
# print(f1_score(test_y, clf_dt.predict(test_data_vect), average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
# print(f1_score(test_y, clf_lr.predict(test_data_vect), average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))

test_set = ["kuratti is a good person", "kuratti is a junk person", "dumb person"]
new_test = vectorizer.transform(test_set)
result = clf_svm.predict(new_test)
# print(result)


from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_data_vect,train_y)
# print(f1_score(test_y, clf.predict(test_data_vect), average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
# import pickle
# # with open('./Models/sentiment_classifier.pkl', 'wb') as f:
# # 	pickle.dump(clf, f)
# with open('./Models/vector.pkl', 'rb') as f:
# 	vector = pickle.load(f)
# test_list = ["Im bad"]
# test_data = vector.transform(test_list)
# with open('./Models/sentiment_classifier.pkl', 'rb') as f:
# 	loaded_pkl = pickle.load(f)

# print(clf.predict(test_data)[0])




GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [6]:
print(clf.predict(test_data_vect[0]))

['NEGATIVE']
